In [1]:
!pip install pandas pymongo

In [3]:
import os
import pandas as pd
from pymongo import MongoClient

In [4]:
client = MongoClient(port=27017)
db = client["bank_project"]
collection = db["economic_indicators"]

In [5]:
# Path to the folder 
excel_folder_path = "GemDataEXTR"  

In [9]:
# Function to check if file is relevant
def is_relevant_file(filename):
    return ('GDP' in filename.upper() or 'CPI' in filename.upper()) and filename.endswith('.xlsx')


In [11]:
# Clear existing collection to start fresh
collection.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [13]:
# Process files with GDP or CPI in their names
processed_files = []
for file_name in os.listdir(excel_folder_path):
    if is_relevant_file(file_name):
        file_path = os.path.join(excel_folder_path, file_name)
        print(f"Processing file: {file_name}")
        
        try:
            # Load the Excel file
            df = pd.read_excel(file_path)
            
            # Add metadata to each record
            data = df.to_dict(orient="records")
            for record in data:
                record['indicator_type'] = 'GDP' if 'GDP' in file_name.upper() else 'CPI'
                record['file_source'] = file_name
                record['seasonally_adjusted'] = 'seas. adj.' in file_name.lower()
            
            # Insert into MongoDB
            collection.insert_many(data)
            processed_files.append(file_name)
            print(f"Successfully inserted {len(data)} records from {file_name}")
            
        except Exception as e:
            print(f"Error processing {file_name}: {str(e)}")

print("\nProcessed Files:")
for file in processed_files:
    print(f"- {file}")

Processing file: Core CPI, not seas. adj..xlsx
Successfully inserted 31 records from Core CPI, not seas. adj..xlsx
Processing file: Core CPI, seas. adj..xlsx
Successfully inserted 31 records from Core CPI, seas. adj..xlsx
Processing file: CPI Price, % y-o-y, median weighted, seas. adj..xlsx
Successfully inserted 31 records from CPI Price, % y-o-y, median weighted, seas. adj..xlsx
Processing file: CPI Price, % y-o-y, nominal, seas. adj..xlsx
Successfully inserted 31 records from CPI Price, % y-o-y, nominal, seas. adj..xlsx
Processing file: CPI Price, nominal, not seas. adj..xlsx
Successfully inserted 31 records from CPI Price, nominal, not seas. adj..xlsx
Processing file: CPI Price, nominal, seas. adj..xlsx
Successfully inserted 31 records from CPI Price, nominal, seas. adj..xlsx
Processing file: GDP at market prices, constant 2010 LCU, millions, seas. adj..xlsx
Successfully inserted 31 records from GDP at market prices, constant 2010 LCU, millions, seas. adj..xlsx
Processing file: GDP 

In [15]:
# Display sample of the data
print("\nSample records from database:")
for record in collection.find().limit(2):
    print(record)


Sample records from database:
{'_id': ObjectId('67551105babcc9a9ae100496'), 'Unnamed: 0': nan, 'Albania': nan, 'Armenia': nan, 'Belgium': nan, 'Belarus': nan, 'Brazil': nan, 'Canada': nan, 'Switzerland': nan, 'Chile': nan, 'China': nan, 'Cameroon': nan, 'Colombia': nan, 'Costa Rica': nan, 'Cyprus': nan, 'Czech Republic': nan, 'Germany': nan, 'Denmark': nan, 'Dominican Republic': nan, 'Ecuador': nan, 'Egypt, Arab Rep.': nan, 'Spain': nan, 'Fiji': nan, 'France': nan, 'United Kingdom': nan, 'Georgia': nan, 'Greece': nan, 'Guatemala': nan, 'Hong Kong SAR, China': nan, 'Honduras': nan, 'Croatia': nan, 'Hungary': nan, 'Indonesia': nan, 'India': nan, 'Ireland': nan, 'Iraq': nan, 'Iceland': nan, 'Israel': nan, 'Italy': nan, 'Jordan': nan, 'Japan': nan, 'Kyrgyz Republic': nan, 'Korea, Rep.': nan, 'Kuwait': nan, 'Lao, PDR': nan, 'Sri Lanka': nan, 'Lithuania': nan, 'Luxembourg': nan, 'Latvia': nan, 'Morocco': nan, 'Moldova, Rep.': nan, 'Mexico': nan, 'North Macedonia': nan, 'Malta': nan, 'Maurit

In [21]:
def explore_data():
    # Count records by indicator type
    gdp_count = collection.count_documents({"indicator_type": "GDP"})
    cpi_count = collection.count_documents({"indicator_type": "CPI"})
    
    print(f"Total GDP records: {gdp_count}")
    print(f"Total CPI records: {cpi_count}")
    
    # Get list of all countries
    sample_record = collection.find_one({"indicator_type": "GDP"})
    countries = [key for key in sample_record.keys() 
                if key not in ['_id', 'Unnamed: 0', 'indicator_type', 'file_source', 'seasonally_adjusted']]
    
    print(f"\nNumber of countries in dataset: {len(countries)}")
    print("\nSample countries:", countries[:5])
    
    # Check date range - Fixed version
    years_data = list(collection.distinct("Unnamed: 0"))
    # Convert years to numeric values, filtering out any non-numeric entries
    years = [float(year) for year in years_data if str(year).replace('.', '').isdigit()]
    
    if years:  # Make sure we have valid years
        min_year = min(years)
        max_year = max(years)
        print(f"\nDate range: {int(min_year)} to {int(max_year)}")
    else:
        print("\nNo valid years found in the dataset")

explore_data()

Total GDP records: 245
Total CPI records: 186

Number of countries in dataset: 91

Sample countries: ['Albania', 'United Arab Emirates', 'Argentina', 'Australia', 'Austria']

Date range: 1995 to 2024


In [25]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from datetime import datetime

def clean_economic_data():
    # Connect to MongoDB
    client = MongoClient(port=27017)
    db = client["bank_project"]
    collection = db["economic_indicators"]
    
    # Convert MongoDB data to DataFrame for easier cleaning
    # We use list() to ensure we get all documents from the cursor
    raw_data = list(collection.find())
    if not raw_data:
        print("No data found in the collection")
        return None
    
    data = pd.DataFrame(raw_data)
    
    # First, let's handle the date column more carefully
    try:
        # Convert 'Unnamed: 0' to numeric, handling any non-numeric values
        data['date'] = pd.to_numeric(data['Unnamed: 0'], errors='coerce')
        # Convert to datetime, assuming years as input
        data['date'] = pd.to_datetime(data['date'].astype(str), format='%Y', errors='coerce')
    except Exception as e:
        print(f"Error processing dates: {str(e)}")
        # Keep original values if conversion fails
        data['date'] = data['Unnamed: 0']

    # Identify country columns more robustly
    exclude_cols = ['_id', 'Unnamed: 0', 'indicator_type', 
                   'file_source', 'seasonally_adjusted', 'date']
    country_cols = [col for col in data.columns if col not in exclude_cols]

    # Handle missing values and data conversion
    for col in country_cols:
        try:
            # Convert to numeric, replacing any non-numeric values with NaN
            data[col] = pd.to_numeric(data[col], errors='coerce')
            
            # Group by indicator type and interpolate missing values
            data[col] = data.groupby('indicator_type')[col].transform(
                lambda x: x.interpolate(method='linear', limit_direction='both')
            )
        except Exception as e:
            print(f"Error processing column {col}: {str(e)}")

    # Create a new cleaned collection with error handling
    try:
        cleaned_collection = db["cleaned_economic_data"]
        # Convert DataFrame to records for MongoDB
        records = data.to_dict('records')
        
        # Clear existing data if any
        cleaned_collection.delete_many({})
        
        # Insert new cleaned data
        if records:
            cleaned_collection.insert_many(records)
            print(f"Successfully inserted {len(records)} cleaned records")
        else:
            print("No records to insert")
            
    except Exception as e:
        print(f"Error saving to MongoDB: {str(e)}")
        return data

    return data

# Run the cleaning process with error handling
try:
    cleaned_data = clean_economic_data()
    if cleaned_data is not None:
        print("Data cleaning completed successfully")
        # Display some basic information about the cleaned data
        print(f"Cleaned data shape: {cleaned_data.shape}")
        print("\nSample of cleaned data:")
        print(cleaned_data.head())
except Exception as e:
    print(f"An error occurred during data cleaning: {str(e)}")

Error saving to MongoDB: NaTType does not support utcoffset
Data cleaning completed successfully
Cleaned data shape: (431, 182)

Sample of cleaned data:
                        _id Unnamed: 0   Albania   Armenia   Belgium  \
0  67551105babcc9a9ae100496        NaN  97.48328  80.22518  83.16487   
1  67551105babcc9a9ae100497     1995.0  97.48328  80.22518  83.16487   
2  67551105babcc9a9ae100498     1996.0  97.48328  80.22518  83.16487   
3  67551105babcc9a9ae100499     1997.0  97.48328  80.22518  84.02403   
4  67551105babcc9a9ae10049a     1998.0  97.48328  80.22518  85.13964   

    Belarus    Brazil    Canada  Switzerland     Chile  ...  Venezuela, RB  \
0  48.49429  37.98033  79.53256     96.81781  99.99093  ...       60.06006   
1  48.49429  37.98033  79.53256     96.81781  99.99093  ...       60.06006   
2  48.49429  44.82968  80.71983     96.81781  99.99093  ...       60.06006   
3  48.49429  48.07538  81.96685     96.81781  99.99093  ...       60.06006   
4  48.49429  49.37448  8

In [37]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

def clean_economic_data():
    # Connect to MongoDB
    client = MongoClient(port=27017)
    db = client["bank_project"]
    collection = db["economic_indicators"]
    
    # Get raw data
    raw_data = list(collection.find())
    print(f"Found {len(raw_data)} documents in original collection")
    
    # Convert to DataFrame
    data = pd.DataFrame(raw_data)
    print(f"Created DataFrame with shape: {data.shape}")
    
    try:
        # Instead of converting dates, let's keep the original year values
        # We'll just clean them up a bit
        data['year'] = data['Unnamed: 0']
        
        # Remove problematic columns we don't need
        columns_to_keep = [col for col in data.columns 
                         if col not in ['_id', 'Unnamed: 0', 'date']]
        data = data[columns_to_keep]
        
        # Handle numeric conversion for country columns
        for col in data.columns:
            if col not in ['year', 'indicator_type', 'file_source', 'seasonally_adjusted']:
                # Convert to float instead of int to handle missing values better
                data[col] = pd.to_numeric(data[col], errors='coerce')
        
        print("Completed data processing")
        
        # Convert to MongoDB format
        try:
            cleaned_collection = db["cleaned_economic_data"]
            cleaned_collection.delete_many({})
            
            # Convert DataFrame to records, replacing NaN with None
            records = data.replace({np.nan: None, np.inf: None, -np.inf: None}).to_dict('records')
            
            if records:
                cleaned_collection.insert_many(records)
                print(f"Successfully inserted {len(records)} documents")
            
        except Exception as e:
            print(f"Error during MongoDB insertion: {str(e)}")
            return data
        
        return data
        
    except Exception as e:
        print(f"Error during data processing: {str(e)}")
        return None

# Run cleaning process
cleaned_data = clean_economic_data()

# Verify the results
if cleaned_data is not None:
    print("\nCleaning completed successfully")
    
    # Let's look at what we have
    def analyze_cleaned_data():
        client = MongoClient(port=27017)
        db = client["bank_project"]
        cleaned_collection = db["cleaned_economic_data"]
        
        count = cleaned_collection.count_documents({})
        print(f"\nNumber of documents in cleaned collection: {count}")
        
        if count > 0:
            sample = cleaned_collection.find_one()
            print("\nSample data fields:")
            for key, value in sample.items():
                print(f"- {key}: {type(value).__name__}")
            
            # Show some basic statistics
            print("\nNumber of documents by indicator type:")
            pipeline = [
                {"$group": {"_id": "$indicator_type", "count": {"$sum": 1}}}
            ]
            for result in cleaned_collection.aggregate(pipeline):
                print(f"- {result['_id']}: {result['count']} documents")
    
    analyze_cleaned_data()
else:
    print("\nCleaning failed")

Found 431 documents in original collection
Created DataFrame with shape: (431, 181)
Completed data processing
Successfully inserted 431 documents

Cleaning completed successfully

Number of documents in cleaned collection: 431

Sample data fields:
- _id: ObjectId
- Albania: NoneType
- Armenia: NoneType
- Belgium: NoneType
- Belarus: NoneType
- Brazil: NoneType
- Canada: NoneType
- Switzerland: NoneType
- Chile: NoneType
- China: NoneType
- Cameroon: NoneType
- Colombia: NoneType
- Costa Rica: NoneType
- Cyprus: NoneType
- Czech Republic: NoneType
- Germany: NoneType
- Denmark: NoneType
- Dominican Republic: NoneType
- Ecuador: NoneType
- Egypt, Arab Rep.: NoneType
- Spain: NoneType
- Fiji: NoneType
- France: NoneType
- United Kingdom: NoneType
- Georgia: NoneType
- Greece: NoneType
- Guatemala: NoneType
- Hong Kong SAR, China: NoneType
- Honduras: NoneType
- Croatia: NoneType
- Hungary: NoneType
- Indonesia: NoneType
- India: NoneType
- Ireland: NoneType
- Iraq: NoneType
- Iceland: Non

In [43]:
from flask import Flask, render_template, jsonify, request
from pymongo import MongoClient

# Create our website's brain
app = Flask(__name__)

# Create a way to talk to our database
def connect_to_database():
    client = MongoClient(port=27017)
    return client["bank_project"]

# Create our homepage route
@app.route('/')
def home():
    return render_template('index.html')

# Create a route to get list of countries
@app.route('/api/countries')
def get_countries():
    db = connect_to_database()
    collection = db["cleaned_economic_data"]
    # Get list of unique countries from our data
    sample = collection.find_one()
    countries = [key for key in sample.keys() 
                if key not in ['_id', 'year', 'indicator_type', 
                             'file_source', 'seasonally_adjusted']]
    return jsonify({"countries": sorted(countries)})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

E:\Soheil\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
